In [1]:
# Utils
import os, sys, re, time, gc, types, string, unicodedata, unidecode, string, warnings, inspect

import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import time
#import _pickle as pickle
import pickle
import re, sys, unidecode
#import unidecode

from sklearn.metrics import accuracy_score
from IPython.display import display, Markdown, Latex

import shutil
import os

# Representation
import nltk
from nltk.corpus import stopwords
import scikitplot as skplt

import wordbatch
from wordbatch.extractors import WordBag, WordHash
from wordbatch.models import FTRL

from tensorflow.contrib.learn import DNNClassifier

#from tecnosmartlib import DataObject

#plt.style.use('fivethirtyeight')

#Carga stop word
nltk.download('stopwords')
spanish_stopwords = nltk.corpus.stopwords.words('spanish')

# punkt:  módulo contiene modelos para la tokenización de textos
nltk.download('punkt')

/root/anaconda3/lib/python3.6/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so | _pywrap_tensorflow_internal
/root/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/lite/toco/python/_tensorflow_wrap_toco.so | _tensorflow_wrap_toco
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Parameters

In [2]:
root_path = "../"
path_data_clean = root_path + "data/clean/"
path_model = root_path + 'models/'
features_path = root_path + 'data/features/'
model_name = "test"
path_model += model_name

delete_old_model = True
if delete_old_model:
    try:
        os.system("rm -rf "+path_model)
        os.system("mkdir "+path_model)
    except:
        print("error")

# Load Data

In [3]:
dfNoticias = pd.read_pickle(path_data_clean + "/dfNoticiasClean.p")


In [4]:
dfNoticias.head()

,Cuerpo,Fecha,Hora,ID,Resumen,Seccion_1,Seccion_2,Seccion_3,Subtema_1,Subtema_2,Subtema_3,Tema_1,Tema_2,Tema_3,Titular,Type
1,\nMarcel Granollers (44°) sorprendió a David F...,20140929,09:57,20140929095927,\nEl español cayó ante su compatriota Marcel G...,deportes,None,None,torneos atp,None,None,tenis,None,None,\nDavid Ferrer sufrió otra temprana eliminación\n,fid_noticia
2,\nEl Gobierno de Barack Obama ha enviado cuatr...,20140929,07:03,20140929065051,\nLas conversaciones comenzaron el año 2010 y ...,pais,mundo,mundo,eeuu,relaciones exteriores,None,relaciones exteriores,eeuu,cuba,\nObama ha enviado cuatro solicitudes a Chile ...,fid_noticia
3,\nEste miércoles la Comisión Asesora Presidenc...,20140929,11:37,20140929105234,\nRepresentantes del sector privado acusaron q...,pais,None,None,isapre,None,None,salud,None,None,\nQuiebre en comisión presidencial de isapres ...,fid_noticia
4,"\nWolfgang Burmann, del Team Avanti, se adjudi...",20140929,10:05,20140929100825,\nEl pedalero del equipo Avanti terminó en el ...,deportes,None,None,chilenos,None,None,ciclismo,None,None,\nWolfgang Burmann ganó el segundo clasificato...,fid_noticia
6,"\nEl Gobierno, a través de Sernapesca, present...",20140929,15:47,20140929152501,\nBachelet instruyó a los ministros de Economí...,pais,pais,None,None,None,None,medioambiente,region de valparaiso,None,\nGobierno presentó querella por derrame de pe...,fid_noticia


# pre procesing



In [5]:
_map = {}
_map["Cuerpo"] = []
_map["Seccion"] = []
_map["Subtema"] = []

count = 0
for index, row in dfNoticias.iterrows():    
    _map["Cuerpo"].append(row["Cuerpo"]) 
    _map["Seccion"].append(row["Seccion_1"])
    _map["Tema"].append(row["Tema_1"])
    _map["Subtema"].append(row["Subtema_1"])

df = pd.DataFrame(_map)
# Elimino clase corporativo, muy pocos ejemplos
df = df[df.Seccion != "corporativo"]

    
X_untransformed = df['Cuerpo'].reset_index(drop=True)
y1_untransformed = df['Seccion'].reset_index(drop=True)
y2_untransformed = df['Tema'].reset_index(drop=True)
y3_untransformed = df['Subtema'].reset_index(drop=True)


# Feature Extraccion

### Data representation
* Normalisacion
* TFID calculation

In [6]:
from nltk.stem.snowball import SnowballStemmer

def normalize_text(text):
    """ Funcion de normalizacion """    
    # split into words
    tokens = nltk.tokenize.word_tokenize(text,language='spanish', preserve_line=False)
    # convert to lower case
    tokens = [w.lower() for w in tokens]    
    
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    
    # remove remaining tokens that are n<<<<<<<<<<<<<<<<<<<<<
    words = [word for word in stripped if word.isalpha()]
    
    # stop word and remove accent
    def strip_accents(s):
        return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
    stop_words = set(spanish_stopwords)
    words = [strip_accents(w) for w in words if not w in stop_words]
    
    stemmer = SnowballStemmer("spanish")
    out = ""
    for word in words:
        out += stemmer.stem(word)+" "
    
    return out
    
#   return u" ".join(words)

# spanish_stopwords = stopwords.words('spanish')
# def normalize_text(text):
#     return u" ".join([x for x in [y for y in text.lower().strip().split(" ")] 
#                       if len(x) > 1 and x not in spanish_stopwords])

In [8]:
 # true calcula features, false carga si ya estan calculadas 272 old time


start = time.time()
print("start")


X_untransformed = X_untransformed
n_docs = X_untransformed.shape[0]
n_cpu = 24

batch_size = int(n_docs/n_cpu)

#'log', "idf":50.0
wb = wordbatch.WordBatch(normalize_text, 
                         extractor=(WordBag, {"hash_ngrams": 1, "hash_ngrams_weights": [1.0, 1.0],
                                              "hash_size": 2**28, "norm": "l2", "tf": 1.0,
                                              "idf": 1.0}), procs=n_cpu, n_words=1000, minibatch_size=batch_size)

wb.dictionary_freeze = True
word_comment = wb.fit_transform(list(X_untransformed),reset= False)
# revisar esta normalizacion
X_transformed = word_comment[:, np.array(np.clip(word_comment.getnnz(axis=0) - 1, 0, 1), dtype = bool)]

end = time.time()
print("TFIDF end time :" + str(end - start) )

X = X_transformed

#X = X.todense()
print('Number of features: {}'.format(X.shape[1]))



start
Normalize text
Extract wordbags
TFIDF end time :229.15021920204163
Number of features: 1000


### Output data set train/test and save

In [9]:
y1 = y1_untransformed.values

np.random.seed(42)

train_fraction = 0.8

train_indices = np.random.choice(X.shape[0], round(train_fraction*X.shape[0]), replace=False)
test_indices = np.array(list(set(range(X.shape[0])) - set(train_indices)))

X_train = X[train_indices]
y_train = y[train_indices]
X_test = X[test_indices]
y_test = y[test_indices]



In [10]:

features_path_ = features_path + "dataset_tfid_hash28_n1000.p"

pickle.dump( (X_train,y_train,X_test,y_test) , open( features_path_, "wb" ) )


# with open("features_path", "wb") as f:
#     pickle.dump(, f)

# with open("features_path", "rb") as f:
#     X_train,y_train,X_test,y_test = pickle.load(f)

In [23]:
wb.transform(['asd auto casa'])

Normalize text
Extract wordbags


<1x268435456 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [25]:
X_train,y_train,X_test,y_test = pickle.load( open( features_path_, "rb" ) )